In [ ]:
import pandas as pd
import numpy as np
#merge and join files
import glob
import os
#regex
import re

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## A- Clean kaggle data (only US and France):

In [ ]:
kaggle_tweets = pd.read_csv('../data/tweets_us/kaggle_en_tweets/vaccination_all_tweets.csv')

In [ ]:
kaggle_tweets.shape

In [ ]:
kaggle_tweets.head(2)

In [ ]:
kaggle_tweets = kaggle_tweets.dropna(subset = ['user_location'])

### 1- Rename and drop columns:

In [ ]:
kaggle_tweets = (kaggle_tweets
          .drop(columns = ['user_created', 'user_favourites', 'source', 'favorites', 'is_retweet'])
          .rename(columns = {'id': 'tweet_id'})
         )

### 2- Keep only tweets in France and USA:

In [ ]:
# Get a list of user location (higher count on top)  
#list_cities = kaggle_tweets.groupby('user_location')['user_location'].count().sort_values(ascending=False).to_frame()

In [ ]:
# Make lists of places of interest for the study (France, south and north of USA)
cities_fr = ['France', 'Paris, France', 'Nancy, France', 'Dammam, KSA | Tunisia | France', 'Alsace, France', 'Paris', 'La Belle Province', 'Bordeaux, France', 'Alpes-Maritimes, Provence-Alpe', '#CoronavirusMadeInPasteur', 'Rome, Italy - Beaulieu, France', 'Europe (usually Poland/France)', 'Paris, Limoges. France']
cities_us_north = ['La Crescenta-Montrose, CA', 'San Francisco, CA', 'Los Angeles, CA', 'Rochester, NY', 'Cranbury, NJ', 'USA🇺🇸', 'America', 'Medford, OR', 'LI, NYC, NJ, PA, NE, USA', 'Chicago, IL', 'Seattle, WA',  'New Jersey, USA', 'Oregon, USA', 'California, USA', 'New York, NY', 'Cambridge, MA', 'New Orleans, LA', 'Michigan', 'Philadelphia, Pa.', 'North America', 'Troy, Michigan;  Serial ✈️', 'Pennsylvania, USA', 'Saugerties, New York', 'Balboa Peninsula, SoCal USA', 'New York / New Jersey / Global', 'Minneapolis, MN', 'Vermont, US', 'Naperville, IL', 'Sacramento', 'New York, USA', 'Cherry Hill, NJ', 'NYC', 'San Jose, CA (USA)', 'Colorado Springs, Colorado ', 'Silicon Valley, CA', 'Washington, DC', 'D.C By Way Of BOSTON!🇳🇬', 'New York', 'Mequon, Wisconsin', 'Boston, MA', 'New York City, Westport, CT', 'Michigan, USA', 'US - IL - ID', 'Washington DC', 'CHICAGO!', 'NY', 'Brooklyn, NY', 'Los Angeles', 'Providence, RI', 'Boston', 'New York City', 'Rancho Cucamonga, CA', 'Syracuse, NY', 'New York & San Francisco', 'Philadelphia, PA', 'New Jersey', 'Buffalo, NY', 'Connecticut, USA', 'Rhode Island, USA', 'Northern Illinois', 'Rochester NY', 'Southern CA', 'Staten Island, New York', 'South Lake Tahoe, CA', 'Boston, MA.', 'washington', 'Paramus, NJ', 'Orange County, CA', 'Elmhurst, IL', 'Grand Rapids, MI', 'Long Beach, CA', 'Washington, D.C.', 'Northern California, USA', 'Princeton, NJ', 'Ventura, CA ', 'Washington D.C.', 'Rochester, MN', 'Denver, CO', 'Englewood, NJ', 'Katonah, NY', 'Chicago', 'Valencia,CA', 'Bellevue, WA', 'Minnesota, USA', 'Old Mystic, CT USA', 'San Francisco Bay Area', 'Santa Barbara, CA', 'Baltimore', 'Vermont, USA', 'Sunnyvale, CA', 'Bethesda, MD.', 'Queens, NY', 'Rochester MN', 'Social distancing in Manhattan', 'Morristown, NJ', 'Bronx, NY', 'DC Metro Area, USA', 'Maryland, USA', 'Stony Brook,  NY', 'Bryn Mawr, PA', 'Madison, WI', 'Coventry Rhode Island', 'Ann Arbor, ✋', 'Colorado, USA', 'Baltimore, MD', 'Pittsburgh, PA', 'Wisconsin, USA', 'minneapolis', 'Southern California', 'Northern California', 'Potomac  MD', 'Seaside, California, USA', 'Hartford, CT', 'Long Island   ', 'California', 'Paterson, NJ', 'NH', 'Manhattan, NYC', 'California Bay Area', 'VT', 'Stanford, CA', 'Duarte, CA', 'Los Angeles, California', 'Central California', 'Portland, ME', 'Philadelphia', 'Riverdale Park, MD ', 'Manhattan, NY', 'East Hampton, NY', 'Hollywood, CA, USA', 'South Cal', 'Ann Arbor, MI', 'Batavia, IL', 'Cambridge, MA, United States', 'Chicago-adjacent, IL', 'Metro North of Boston MA', 'New Haven, CT', 'Silver Spring, MD', 'West Hollywood, CA', 'So Cal', 'SoCal', 'California, United States', 'Sacramento, CA', 'Massachusetts, USA', 'Illinois, USA', 'Washington, USA', 'Fremont, CA, USA', 'New Jersey', 'Portland, OR', 'San Francisco', 'New York, New York', 'Santa Monica, CA', 'Connecticut', 'West Hartford, CT', 'Oakland, CA', 'Anaheim, CA', 'San Jose, CA', 'Buffalo, NY', 'Massachusetts', 'Los Angeles, California', 'Vermont, USA', 'Detroit, MI', 'Delaware, USA', 'Philadelphia', 'Boothbay Harbor, Maine', 'San Diego, California']
cities_us_south = ['Texas', 'Jefferson City, MO',  'LA | OC ', 'Beaumont, Texas', 'Kentucky', 'Dallas, TX', 'Lakeland, FL', 'Palm Beach, FL', 'Tampa, FL', 'Okolona, KY', 'Houston, TX', 'Miami, FL', 'McAllen, TX', 'Texas, USA', 'Miami', 'Charleston, SC', 'Florida, USA', 'West Virginia, USA', 'Raleigh, NC', 'Raleigh, NC, USA', 'South Carolina, USA', 'Durham, NC', 'Alabama', 'Cincinnati, OH ', '11150A Asheville Hwy Inman, SC', 'Tucson, AZ', 'Albuquerque, NM', 'Nashville, TN', 'St Petersburg, FL', 'Austin, TX', 'Kansas', 'Roanoke, VA', 'Clinton, SC', 'Miami Beach, FL', 'San Antonio, Texas', 'Atlanta, GA', 'Odessa, TX', 'Chapel Hill, NC', 'Scottsdale, AZ', 'NEVADA', 'Richmond, VA', 'Atlanta, Georgia', 'Tysons Corner, Virginia', 'Phoenix, AZ', 'Greensboro, NC', 'Las Vegas, NV', 'Harlingen, TX', 'North Carolina, USA', 'Indianapolis, IN', 'Spring, TX', 'Idaho, USA', 'Athens, GA', 'Jacksonville, FL', 'Houston Texas Based', 'Jackson, MS', 'Covid Hell, AL', 'Kansas City, KS', 'Fort Worth', 'Charlottesville, VA', 'Kansas City, MO', 'Florida', 'Dayton, OH', 'Saint Louis, MO, USA', 'St Louis, MO', 'Knoxville, TN', 'Birmingham, AL', 'Columbia, MO', 'Morgantown, WV', 'TX', 'Allen, TX', 'Arkansas', 'Gainesville, FL', 'Smithville, TX', 'Augusta, GA', 'West Texas', 'Oklahoma City & Tulsa, OK', 'Oklahoma, USA', 'Gainesville, FL, USA', 'Arkansas, United States', 'Sugar Land, TX', 'Miami, Fl', 'Central FL', 'arkansas', 'Riverdale, UT', 'Nashville', 'Dallas', 'Houston', 'Reno, NV', 'Tampa Bay, FL', 'Charlotte, NC', 'Austin', 'Kansas City ', 'South Florida', 'Salt Lake City, UT', 'Alabama, USA', 'Ohio, USA', 'Virginia, USA', 'Orlando, FL', 'Las Vegas', 'Lufkin, TX', 'Arizona, USA', 'Cleveland, OH', 'Georgia, USA', 'Fort Lauderdale, FL', 'New York, Sarasota, FL', 'Columbus, OH', 'Louisville, KY', 'Tennessee, USA', 'Missouri, USA', 'Indiana, USA', 'Kentucky, USA', 'Oklahoma City, OK', 'North Carolina', 'New Mexico, USA', 'Rio Grande Valley, TX', 'Houston, Tx', 'Ohio', 'Cleveland, Ohio', 'Utah, USA', 'Indiana', 'Salt Lake City, Utah', 'Dallas, Texas', 'South Carolina', 'Columbus, Ohio', 'Corpus Christi, TX', 'Tampa, FL - United States', 'Atlanta', 'Indianapolis', 'Salt Lake City, UT', 'Alabama', 'TX', 'Fort Worth, TX', 'Jacksonville, N.C.', 'Tennessee', 'Arkansas, USA', 'Baton Rouge, LA', 'Arkansas', 'Jupiter, FL', 'Louisiana, USA', 'Tulsa, OK']

In [ ]:
# Make dfs with the list of cities
tweets_fr = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_fr)]
tweets_us_north = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_north)]
tweets_us_south = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_south)]

## B- Clean data from Tweepy:

### 1- France:

**Merge files to one file and add a column with the respective vaccines names(week 2-3):**

In [ ]:
# merging astrazeneca files
astra_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/astra/tweets_fr_tweets_fr_aztra_2021_05_18_to_05_12.csv', '../data/tweets_fr/astra/tweets_fr_tweets_fr_astra_2021_05_19_to_05_13.csv']), ignore_index=True)
astra_fr.shape

In [ ]:
# Make a new column with vaccine name
astra_fr['vaccine'] = 'astrazeneca'

In [ ]:
# merging J&J files
jandj_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_19_to_05_14.csv', '../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_18_to_05_11.csv']), ignore_index=True)
jandj_fr.shape

In [ ]:
# Make a new column with vaccine name
jandj_fr['vaccine'] = 'johnson'

In [ ]:
# merging pfizer files
pfizer_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_14_to_05_11.csv', '../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_18_to_05_15.csv']), ignore_index=True)
pfizer_fr.shape

In [ ]:
# Make a new column with vaccine name
pfizer_fr['vaccine'] = 'pfizer'

In [ ]:
# moderna
moderna_fr = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_18_to_05_11.csv')
moderna_fr['vaccine'] = 'moderna'

In [ ]:
# Concatenate dataframes together
vaccines_fr = pd.concat([astra_fr, jandj_fr, pfizer_fr, moderna_fr], ignore_index=True)

In [ ]:
vaccines_fr.shape

In [ ]:
# Get tweets with a different search
all_fr = pd.read_csv('../data/tweets_fr/tweets_fr_all_2021_05_19_to_05_11.csv')

In [ ]:
all_fr.shape

In [ ]:
#columns = ["Tweet Id", "Tweet Date", "location", "Follower Count", "Account Verified", "Favorite Count", "Retweets", "Tweet Text", "hashtags"]

In [ ]:
# Get the tweets ID that are in common with the 2 different searchs
common = all_fr.merge(vaccines_fr, how='inner', on=["Tweet Id"]) 

In [ ]:
common.shape

In [ ]:
# Get the tweets that are not in vaccine_fr
all_fr_no_com = all_fr[(~all_fr["Tweet Id"].isin(common["Tweet Id"]))]
all_fr_no_com.shape

In [ ]:
# Add a column with the respective vaccine for each tweet
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

In [ ]:
# Drop rows with NaN values in column "vaccine"
all_fr_no_com = all_fr_no_com.dropna(subset=[9], axis=1)

In [ ]:
# Concatenate all_fr_no_com to vaccines_fr
vaccines_fr = pd.concat([vaccines_fr, all_fr_no_com], ignore_index=True)

In [ ]:
# Rename and drop columns 
vaccines_fr = (vaccines_fr
          .drop(columns = ['Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [ ]:
# change "date" object type to datetime
vaccines_fr['date'] = vaccines_fr['date'].astype('Datetime64')

**Encode 'utf-8' characters with real characters:**

In [ ]:
# use eval to decode "utf-8" and remove "\n"
vaccines_fr["text"] = vaccines_fr["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))

**Removing Twitter Handles (@user) and links:**

In [ ]:
def remove_pattern(input_text, pattern):
    reg = re.findall(pattern, input_text)
    for x in reg:
        input_text = re.sub(x, '', input_text)
        
    return input_text

In [ ]:
# twitter handles
vaccines_fr["text"] = np.vectorize(remove_pattern)(vaccines_fr['text'], "@[\w]*")

In [ ]:
# links
vaccines_fr["text"] = np.vectorize(remove_pattern)(vaccines_fr['text'], "https://[\w\.\\/]*")

**Save "vaccine_fr" as a csv file:**

In [ ]:
#vaccines_fr.to_csv('../data/vaccines_fr.csv', index=False)

**Merge files to one file and add a column with the respective vaccines names(week 4):**

In [ ]:
pfizer_fr_week4 = pd.read_csv('../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_29_to_05_22.csv')
pfizer_fr_week4['vaccine'] = 'pfizer'
moderna_fr_week4 = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_29_to_05_22.csv')
moderna_fr_week4['vaccine'] = 'moderna'
astra_fr_week4 = pd.read_csv('../data/tweets_fr/astra/tweets_fr_astra_2021_05_29_to_05_22.csv')
astra_fr_week4['vaccine'] = 'astrazeneca'
janssen_fr_week4 = pd.read_csv('../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_29_to_05_22.csv')
janssen_fr_week4['vaccine'] = 'johnson'

In [ ]:
# Concatenate all_fr_no_com to vaccines_fr
vaccines_fr_week4 = pd.concat([pfizer_fr_week4, moderna_fr_week4, astra_fr_week4, janssen_fr_week4], ignore_index=True)

In [ ]:
# Rename and drop columns 
vaccines_fr_week4 = (vaccines_fr_week4
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [ ]:
# change "date" object type to datetime
vaccines_fr_week4['date'] = vaccines_fr_week4['date'].astype('Datetime64')

In [ ]:
# use eval to decode "utf-8" and remove "\n"
vaccines_fr_week4["text"] = vaccines_fr_week4["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
vaccines_fr_week4["text"] = np.vectorize(remove_pattern)(vaccines_fr_week4['text'], "@[\w]*")
# remove links
vaccines_fr_week4["text"] = np.vectorize(remove_pattern)(vaccines_fr_week4['text'], "https://[\w\.\\/]*")

In [ ]:
#Save file
#vaccines_fr_week4.to_csv('../data/vaccines_fr_week4.csv', index=False)

### 2- USA:

**NORTHERN STATES:**

In [ ]:
# Concat two csv files for week 3 and 4 of May
pfizer_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/pfizer/tweets_us_north_pfizer_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/pfizer/tweets_us_north_pfizer_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
pfizer_north['vaccine'] = 'pfizer'

In [ ]:
# Concat two csv files for week 3 and 4 of May
moderna_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/moderna/tweets_us_north_moderna_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/moderna/tweets_us_north_moderna_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
moderna_north['vaccine'] = 'moderna'

In [ ]:
# Concat two csv files for week 3 and 4 of May
janssen_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/JandJ/tweets_us_north_jandj_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/JandJ/tweets_us_north_jandj_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
janssen_north['vaccine'] = 'johnson'

In [ ]:
# Concat two csv files for week 3 and 4 of May
astra_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/astra/tweets_us_north_astra_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/astra/tweets_us_north_astra_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
astra_north['vaccine'] = 'astrazeneca'

In [ ]:
# Concatenate dataframes together
tweets_us = pd.concat([astra_north, janssen_north, pfizer_north, moderna_north], ignore_index=True)

In [ ]:
# Concat two csv files for week 3 and 4 of May
all_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/tweets_us_north_all_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/tweets_us_north_all_2021_05_31_to_05_23.csv']), ignore_index=True)

In [ ]:
# Get the tweets ID that are in common with the 2 different searchs
common_us_n = all_north.merge(tweets_us, how='inner', on=["Tweet Id"])
common_us_n.shape

In [ ]:
# Get the tweets that are not in tweets_us
all_us_n_no_com = all_fr[(~all_north["Tweet Id"].isin(common["Tweet Id"]))]
all_us_n_no_com.shape

In [ ]:
# Add a column with the respective vaccine for each tweet
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

In [ ]:
all_us_n_no_com = all_us_n_no_com.dropna(subset=['vaccine'])

In [ ]:
# Concatenate all_fr_no_com to vaccines_fr
tweets_us_north = pd.concat([tweets_us, all_us_n_no_com], ignore_index=True)

In [ ]:
# Rename and drop columns 
tweets_us_north = (tweets_us_north
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [ ]:
# change "date" object type to datetime
tweets_us_north['date'] = tweets_us_north['date'].astype('Datetime64')

In [ ]:
# use eval to decode "utf-8" and remove "\n"
tweets_us_north["text"] = tweets_us_north["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
tweets_us_north["text"] = np.vectorize(remove_pattern)(tweets_us_north['text'], "@[\w]*")
# remove links
tweets_us_north["text"] = np.vectorize(remove_pattern)(tweets_us_north['text'], "https://[\w\.\\/]*")

In [ ]:
#Save file
#tweets_us_north.to_csv('../data/tweets_us_north.csv', index=False)

**SOUTHERN STATES:**

In [ ]:
# merging the files pfizer
joined_files = os.path.join("../data/tweets_us/us_south/pfizer/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
pfizer_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
pfizer_south['vaccine'] = 'pfizer'

In [ ]:
# merging the files moderna
joined_files = os.path.join("../data/tweets_us/us_south/moderna/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
moderna_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
moderna_south['vaccine'] = 'moderna'

In [ ]:
# merging the files janssen
joined_files = os.path.join("../data/tweets_us/us_south/JandJ/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
janssen_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
janssen_south['vaccine'] = 'johnson'

In [ ]:
# merging the files astra
joined_files = os.path.join("../data/tweets_us/us_south/astra/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
astra_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
astra_south['vaccine'] = 'astrazeneca'

In [ ]:
# Concatenate dataframes together
tweets_us_south = pd.concat([astra_north, janssen_north, pfizer_north, moderna_north], ignore_index=True)

In [ ]:
# Concat two csv files for week 3 and 4 of May
all_south = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_south/tweets_us_all_south_2021_05_22_to_05_16.csv', '../data/tweets_us/us_south/tweets_us_all_south_2021_05_31_to_05_23.csv']), ignore_index=True)

In [ ]:
# at least one tweet was scraped wrongly and the values are not in the correct rows, I decide to drop it
all_south = all_south[all_south["Tweet Id"] != 'He/Him']

In [ ]:
# drop columns with no text ( probably other tweets not scrapped properly)
all_south = all_south.dropna(subset=['Tweet Text'])

In [ ]:
# Add a column with the respective vaccine for each tweet
all_south.loc[all_south["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_south.loc[all_south["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_south.loc[all_south["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_south.loc[all_south["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

In [ ]:
all_south = all_south.dropna(subset=['vaccine'])

In [ ]:
# Concatenate all_fr_no_com to vaccines_fr
tweets_us_south = pd.concat([tweets_us_south, all_south], ignore_index=True)

In [ ]:
# Rename and drop columns 
tweets_us_south = (tweets_us_south
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [ ]:
# change data types
tweets_us_south['date'] = tweets_us_south['date'].astype('Datetime64')
tweets_us_south['id'] = tweets_us_south['id'].astype('int64')
tweets_us_south['follower_count'] = tweets_us_south['follower_count'].astype('int64')

In [ ]:
# use eval to decode "utf-8" and remove "\n"
tweets_us_south["text"] = tweets_us_south["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
tweets_us_south["text"] = np.vectorize(remove_pattern)(tweets_us_south['text'], "@[\w]*")
# remove links
tweets_us_south["text"] = np.vectorize(remove_pattern)(tweets_us_south['text'], "https://[\w\.\\/]*")

In [ ]:
#Save file
#tweets_us_south.to_csv('../data/tweets_us_south.csv', index=False)

## C- Cleaning data from Tweepy round 2:

**FRANCE: Janssen tweets have a lot of noise due to a lot of celebrities having a last name "Johnson" like Boris Johnson for example.**

In [ ]:
# New Janssen search using tweepy
janssen_fr = pd.read_csv('../data/tweets_fr/tweets_fr_jandj_2021_06_03_to_05_27.csv')
janssen_fr['vaccine'] = 'johnson'

In [ ]:
# Rename and drop columns 
janssen_fr = (janssen_fr
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [ ]:
# change "date" object type to datetime
janssen_fr['date'] = janssen_fr['date'].astype('Datetime64')

In [ ]:
# use eval to decode "utf-8" and remove "\n"
janssen_fr["text"] = janssen_fr["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
janssen_fr["text"] = np.vectorize(remove_pattern)(janssen_fr['text'], "@[\w]*")
# remove links
janssen_fr["text"] = np.vectorize(remove_pattern)(janssen_fr['text'], "https://[\w\.\\/]*")

In [ ]:
#Save file
#janssen_fr.to_csv('../data/janssen_fr.csv', index=False)

In [ ]:
# merging files
tweets_fr = pd.concat(
    map(pd.read_csv, ['../data/tweet_fr/tweets_fr_week_2_translated.csv', '../data/tweet_fr/tweets_fr_week4_translated.csv']), ignore_index=True)
tweets_fr.shape

In [ ]:
# get ride of johnson vaccine tweets because the majority are about Boris Johnson (tweepy search wrong)
tweets_fr["boris"] = tweets_fr['text'].str.contains("Boris|boris")
tweets_fr = tweets_fr[tweets_fr.boris == False]

In [ ]:
tweets_fr = tweets_fr.drop(columns = ["boris", "translated_text"])

In [ ]:
# Concatenate dataframes together
tweets_fr = pd.concat([tweets_fr, janssen_fr], ignore_index=True)

In [ ]:
tweets_fr = tweets_fr.drop_duplicates()

In [ ]:
# Get a list of user location (higher count on top)  
#list_cities = tweets_fr.groupby('location')['location'].count().sort_values(ascending=False).to_frame()

In [ ]:
# tweets location with more than 10 tweets not in France
not_france = ['Bruxelles', 'Suisse', 'Brussels', 'Genève, Suisse', 'Bruxelles, Belgique', 'Luxembourg', 'London, England', 'Liège, Belgique', 'Uccle, Belgique', 'Zürich', 'Geneva, Switzerland', 'Saitama\n\nさいたま市\n\n', 'BRUSSELS', 'Namur, Belgium']
tweets_fr = tweets_fr[~tweets_fr['location'].isin(not_france)]

In [ ]:
#save file
#tweets_fr.to_csv('../data/tweets_fr.csv', index=False)

In [ ]:
tweets_fr_trans = pd.read_csv('../data/tweets_fr_translated.csv')

In [ ]:
# There is tweets texts that are duplicates with different tweets id!
tweets_fr_trans.drop_duplicates('text', inplace=True)

In [ ]:
# Remove tweets about campaign of vaccination (about 11% of the data)
tweets_fr_trans['campaign'] = tweets_fr_trans['text'].str.contains("doses sont disponibles|disponibles aujourd'hui|disponible aujourd'hui|Rdv dispo|disponibles demain|disponible demain|doses sont disponibles|créneau disponible|créneaux disponibles|Centre|Centres|Alpexo and CHU")
tweets_fr_trans = tweets_fr_trans[tweets_fr_trans.campaign == False]
tweets_fr_trans = tweets_fr_trans.drop(columns = ["campaign"])

In [ ]:
#save file
#tweets_fr_trans.to_csv('../data/tweets_fr_translated_clean.csv', index=False)

**USA NORTH: Clean location (several tweets are not in NE region):**

In [ ]:
tweets_us_north = pd.read_csv('../data/tweets_us_north.csv')

In [ ]:
# Get a list of user location (higher count on top)  
#list_cities_us_n = tweets_us_north.groupby('location')['location'].count().sort_values(ascending=False).to_frame()

In [ ]:
# remove tweets in location = France
tweets_us_north["france"] = tweets_us_north['location'].str.contains("France|france|FRANCE|Suisse|VA|California|Florida|Bruxelles")
tweets_us_north = tweets_us_north[tweets_us_north.france == False]
tweets_us_north = tweets_us_north.drop(columns = ["france"])

In [ ]:
# remove tweets in french
tweets_us_north["french_tweets"] = tweets_us_north['text'].str.contains("pour|fait|mais|avoir|faire|demain|semaine|aussi|bien|vous|contre|suis|jour|rien")
tweets_us_north = tweets_us_north[tweets_us_north.french_tweets == False]
tweets_us_north = tweets_us_north.drop(columns = ["french_tweets"])

In [ ]:
not_us_n = ['Paris', 'Maryland, USA', 'Washington, USA', 'Baltimore, MD', 'Georgia, USA', 'Marseille', 'Meurthe-et-Moselle', 'Jamaica', 'Maryland', 'New Mexico, USA', 'Essex', 'Nantes', 'Narnia', 'Georgia', 'Strasbourg', 'Lyon', 'Washington', 'Rennes', 'Clermont Ferrand', 'Dorset', 'Lancashire', 'Middle Earth', 'Silver Spring, MD','Baltimore', 'Yorkshire', 'South Wales' , 'Arcturus', 'Paris, Ile-de-France', 'Baltimore, Maryland', 'Idaho, USA', 'Provence-Alpes-Côte d\'Azur', 'Rouen', 'paris', 'Lille', 'Luxembourg', 'London', 'Bordeaux', 'North Wales', 'Toulouse', 'Montpellier', 'Nord-Pas-de-Calais, France', 'FRANCE', 'Paris', 'Idaho', 'Nice', 'Grenoble', 'Brussels', 'Arlington, Virginia', 'Région Parisienne', 'california', 'Bretagne', 'Metz', 'London, England', 'Beaulieu-sur-Mer', 'Dijon', 'Haute-Vienne', 'East Texas', 'Marseille']

In [ ]:
tweets_us_north = tweets_us_north[~tweets_us_north['location'].isin(not_us_n)]

In [ ]:
# There are tweets texts that are duplicates with different tweets id!
tweets_us_north.drop_duplicates('text', inplace=True)

In [ ]:
# Remove tweets about campaign of vaccination (about 15% of the data)
tweets_us_north['campaign'] = tweets_us_north['text'].str.contains("pendant|dans|Vaccine appointments are available at")
tweets_us_north = tweets_us_north[tweets_us_north.campaign == False]
tweets_us_north = tweets_us_north.drop(columns = ["campaign"])

In [ ]:
#save file
#tweets_us_north.to_csv('../data/tweets_us_north-clean.csv', index=False)

**US SOUTH: clean location (some locations are outside of SE region):**

In [ ]:
tweets_us_south = pd.read_csv('../data/tweets_us_south.csv')

In [ ]:
# Get a list of user location (higher count on top)  
list_cities_us_s = tweets_us_south.groupby('location')['location'].count().sort_values(ascending=False).to_frame()

In [ ]:
# location outside of SE region
tweets_us_south["not_s"] = tweets_us_south['location'].str.contains("Northumberland|Massachusetts|Pennsylvania|VA|Virginia|Texas|DE|NY|DC|Ireland|MA|PA|CT|RI|NJ|IL|NH|Washington|California|california|New York|TX|NYC|MD")
tweets_us_south = tweets_us_south[tweets_us_south.not_s == False]
tweets_us_south = tweets_us_south.drop(columns = ["not_s"])

In [ ]:
# tweets location with more than 10 tweets not in SE
not_us_s = ['New Jersey, USA','Pennsylvania, USA', 'Massachusetts, USA', 'Maryland, USA', 'Arizona, USA', 'Connecticut, USA', 'Illinois, USA', 'Minnesota, USA', 'Boston', 'Pennsylvania', 'Lebanon', 'Massachusetts', 'Rhode Island', 'Switzerland', 'Maine, USA', 'Wood River, Illinois 62095', 'NJ', 'Netherlands', 'Jamaica', 'Minnesota', 'Philadelphia', 'North America', 'Maryland', 'Connecticut', 'Houston, Texas', 'Illinois', 'Israel', 'Denmark', 'Utah, USA', 'New Mexico, USA', 'Philadelphia, Pa', 'Utah', 'Maine', 'Sweden', 'Essex', 'Nevada, USA', 'The Moon', 'Narnia', 'Brooklyn', 'nyc', 'Jersey City, NJ', 'Hungary', 'Hindustan', 'Jersey', 'Dorset', 'Rhode Island, USA', 'Lancashire', 'Middle Earth', 'Europa', 'Yorkshire', 'South Wales', 'Arcturus', 'Dallas, Texas', 'Idaho, USA', 'New Hampshire', 'Wilmington, DE', 'Philly', 'Cheshire', 'North Wales', 'Sparta, NJ', 'ireland', 'Idaho', 'Texas, USA', 'Tyler, Texas', 'Maryland', 'new york', 'Dallas', 'Middlebury, VT', 'Massachusetts', 'Dallas County', 'Newfoundland', 'District of Columbia, USA', 'West End, District of Columbia', 'Providence, Rhode Island', 'boston', 'Nevada', 'North', 'Venus', 'Internet', 'Hollywood Hills', 'India', 'Northeast U.S.', 'Oxfordshire', 'E', 'Nevada', 'IRELAND', 'brooklyn', 'Boston, Ma', 'Switzerland']

In [ ]:
tweets_us_south = tweets_us_south[~tweets_us_south['location'].isin(not_us_s)]

In [ ]:
# There is tweets texts that are duplicates with different tweets id!
tweets_us_south.drop_duplicates('text', inplace=True)

In [ ]:
# Remove tweets about campaign of vaccination (about 1.5% of the data)
tweets_us_south['campaign'] = tweets_us_south['text'].str.contains("appointments|clinic|clinics|center|CALL NOW|No appointment needed|to make an appointment|Schedule an appointment|Schedule your appointment|schedule your appointment|no appointment required|No appointment is necessary|patient|no appointment necessary|Center|Mass Vaccination Sites|Find an appointment|Duke Health|UNC Health|receive a free bus pass|Please text|Walk-ins are welcome|No appointment necessary|Walk in")
tweets_us_south = tweets_us_south[tweets_us_south.campaign == False]
tweets_us_south = tweets_us_south.drop(columns = ["campaign"])

In [ ]:
# About 8% are about campaign for vaccination 
tweets_us_south['campaign'] = tweets_us_south['text'].str.contains("appointment")
test= tweets_us_south[tweets_us_south.campaign == True]

In [ ]:
tweets_us_south.shape

In [ ]:
#save file
tweets_us_south.to_csv('../data/tweets_us_south_clean.csv', index=False)